# Getting streamflow data direct from BoM using scripting

Workbook layout:
   1. Get Extended data from BoM Water Data Online and compare to benchmarking data
   

# Get Extended data from BoM Water Data Online

### SOS2 Services for Water Data Online

The Bureau has implemented a Sensor Observation Service standard version 2 (SOS2) service to make publicly available the time series data it collects and manages under the Water Regulations 2008. The service provides data in WaterML2 format. The SOS2 standard was developed by the Open Geospatial Consortium (OGC) as part of the OGC Sensor Web Enablement framework. The WaterML2 standard was also developed by the OGC with support from the Bureau of Meteorology and CSIRO though the WIRADA research alliance.

In [ ]:
%matplotlib inline

import os

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from awrams.utils import config_manager
from awrams.utils.gis import ShapefileDB
import awrams.utils.wiski.interface as wi

In [ ]:
system_profile = config_manager.get_system_profile()
system_settings = system_profile.get_settings()

base_data_path = system_settings['DATA_PATHS']['BASE_DATA']
catchment_shapefile = os.path.join(base_data_path, 'spatial/shapefiles/Final_list_all_attributes.shp')

catchments = ShapefileDB(catchment_shapefile)
records = catchments.get_records_df()

obs_csv = base_data_path + '/observations/runoff/awrams_v5_cal_qobs.csv'
qobs = pd.read_csv(obs_csv, index_col=0)

ids = records['StationID']  # ids as used in awra-l system (often concatenated wrsc)
wrsids = records['WrscID'] # official water resource station catalogue station numbers
areas = records['AlbersArea'] # catchment areas in km^ 2

In [ ]:
#what is the official Water Resources Station Catalogue ID of site '105001'?
wrsids[np.where(ids=='105001')[0][0]]

In [ ]:
# get set of stations (xml elements) that have 'Water Course Discharge'

wcds = wi.get_station_list("Water Course Discharge") # Get discharge volume (m3/sec) from wiski for gauging stations
#### find all datasets for a station "105001"
stns = wi.find_id_in_station_list('105001', wcds) 
stns  #['105001B', '105001A']

In [ ]:
wcds

In [ ]:
# View the types of data you can download usign wiski SOS
wi.WISKI_CODES

### Plot the downloaded data for site '105001B' including quality codes

In [ ]:
siteid = '105001' # site id according to benchmarking dataset
whichsite = np.where(ids==siteid)[0][0] # find which of the set this corresponds to to get the true wrsids ID


# get the data 
a = pd.DataFrame(wi.get_data(wrsids[whichsite],
                 obs_type="Water Course Discharge",
                 period=pd.date_range('1900-01-01', '2017-10-23', freq='D'))) # Setup data to download from Wiski

# plot the data
figsize = (12, 7)
fig = plt.figure(figsize=figsize)
fig.suptitle(wrsids[whichsite] + ' Water Course Discharge [ML/day]')
ax1 = fig.add_subplot(3, 1, 1)
a1 = pd.DataFrame(a['values'])
b = pd.DataFrame(qobs[str(int(ids[whichsite]))])
# Convert mm/day benchmarking data set to cumecs as per downloaded data
# Q(m^3/s) =Q (mm/day) *Catchment Area (m^2) / 1000mm/m * (24*3600second/day)   
# area is provided in km^2
# Q(m^3/s) =Q (mm/day) *Area (km^2)*1000*1000 /1000*24*3600 
#          =Q (mm/day) *Area (km^2)/24*3.6
#          =Q (mm/day) *Area (km^2)/86.4
b = b * areas[whichsite] / 86.4
c = a1.join(b)
dt = pd.to_datetime(a.index, format='%Y-%m-%d')
ax1.plot_date(dt, c['values'], '-', label='Newly downloaded using webservices')
ax1.plot_date(dt, c[siteid], '-', label='Original benchmarking')
ax1.legend()
ax1.set_title('Streamflow [ML/day]')
ax1 = fig.add_subplot(3, 1, 2)
ax1.plot_date(dt, c, '-')  
plt.yscale('log')
ax1.set_title('Streamflow [ML/day] (log-y axis)')

# finaly qualaty codes
ax1 = fig.add_subplot(3, 1, 3)
ax1.plot(a['qualifier'], color='red')
ax1.set_title('BoM Quality codes'+'1034=A, 1114=B, 1134=C, 1164=E, 1234=F')
# see bom quality codes
# currently known issue in SOS with translations of quality codes
#http://www.bom.gov.au/water/wdtf/documentation/schema-control-lists/quality-code.htm
#dictQualityCodes='1034=A, 1114=B, 1134=C, 1164=E, 1234=F'    
 

In [ ]:
areas[whichsite]